# AgERA5 assessment

Production date: DD-MM-2025
**Please note that this repository is used for development and review, so quality assessments should be considered work in progress until they are merged into the main branch.**

Dataset version: 2.0.

Produced by: C3S2_521 contract.

## 🌍 Use case: Use case listed here in full 

## ❓ Quality assessment question
* **In most cases there should be one question listed here in bold**
* **(In some cases a second related/follow-up question may be included)**

A very short introduction before the assessment statement describing approach taken to answer the user question. One or two key references could be useful,  if the assessment summarises literature (referenced directly in the text, or with numerical labels like this (also listed at the end) `[[1]](https://doi.org/10.1038/s41598-018-20628-2))`giving: [[1]](https://doi.org/10.1038/s41598-018-20628-2)).

[[CDS AgERA5]](https://doi.org/10.24381/cds.6c68c9bb).

## 📢 Quality assessment statement

```{admonition} These are the key outcomes of this assessment
:class: note
* Finding 1
* Finding 2
* Finding 3
* etc
```

## 📋 Methodology

**Agrometeorological indicators from 1979 to present derived from reanalysis** (*AgERA5*; [doi 10.24381/cds.6c68c9bb](https://doi.org/10.24381/cds.6c68c9bb)).

A ‘free text’ introduction to the data analysis steps or a description of the literature synthesis, with a justification of the approach taken, and limitations mentioned. **Mention which CDS catalogue entry is used, including a link, and also any other entries used for the assessment**.

* These headings can be specific to the quality assessment, and help guide the user through the ‘story’ of the assessment. This means we cannot pre-define the sections and headings here, as they will be different for each assessment.
* Sub-bullets could be used to outline what will be done/shown/discussed in each section
* The list below is just an example, or may need more or fewer sections, with different headings

The analysis and results are organised in the following steps, which are detailed in the sections below:

**[](section-setup)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-analysis)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-3)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

**[](section-4)**
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.
 
**[](section-5)** 
 * Sub-steps or key points listed in bullet below. No strict requirement to match and link to sub-headings.

Any further notes on the method could go here (explanations, caveats or limitations).

## 📈 Analysis and results

(section-setup)=
### 1. Data and code setup

#### Import required packages
```{note}
This notebook uses [earthkit](https://github.com/ecmwf/earthkit) for 
downloading ([earthkit-data](https://github.com/ecmwf/earthkit-data)) 
and visualising ([earthkit-plots](https://github.com/ecmwf/earthkit-plots)) data.
Because earthkit is in active development, some functionality may change after this notebook is published.
If any part of the code stops functioning, please raise an issue on our GitHub repository so it can be fixed.
```

In [ ]:
import earthkit.data as ekd
import earthkit.plots as ekp
import numpy as np
from matplotlib import pyplot as plt

#### Define request and download
We now define our parameters, i.e. which parts of the dataset we want to download.
This is done in multiple steps so it can be easily adapted to a different use case (time, location, crop, variable).
We first set up the default parameters, which should always be included:

In [ ]:
dataset_ID = "sis-agrometeorological-indicators"

request_default = {
    "version": "2_0",
}

Next, we choose our time parameters, in this case every day in January–September 2024:

In [ ]:
request_time = {
    "year": "2024",
    "month": [f"{mo:02}" for mo in range(1, 10)],
    "day": [f"{d:02}" for d in range(1, 32)],
}

Next, we choose our geographical range, in this example the United Kingdom and Ireland:

In [ ]:
request_domain = {
    "area": [60, -12, 48, 4]  # North, West, South, East
}

Last, we specify the variables of interest:

In [ ]:
request_temperature_max = {
    "variable": "2m_temperature",
    "statistic": ["24_hour_maximum"],
}

request_temperature_min = {
    "variable": "2m_temperature",
    "statistic": ["24_hour_minimum"],
}

request_radiation = {
    "variable": "solar_radiation_flux",
}

request_vapour_mean = {
    "variable": "vapour_pressure",
    "statistic": ["24_hour_mean"],
}

request_wind_mean = {
    "variable": "10m_wind_speed",
    "statistic": ["24_hour_mean"],
}

request_rain = {
    "variable": "precipitation_flux",
}

request_snow = {
    "variable": "snow_thickness",
    "statistic": ["24_hour_mean"],
}

The requests for specific variables are combined with the default, time, and domain parameters and passed to earthkit for download from the CDS:

In [ ]:
def make_full_request(request: dict) -> dict:
    return request_default | request_time | request_domain | request

requests_all_combined = [make_full_request(request) for request in (request_temperature_max, 
                                                                    request_temperature_min, 
                                                                    request_radiation, 
                                                                    request_vapour_mean, 
                                                                    request_wind_mean, 
                                                                    request_rain, 
                                                                    # request_snow,
                                                                   )]

ds = ekd.from_source("cds", dataset_ID, *requests_all_combined)

As a check, we can print the downloaded dataset to inspect its structure:

In [ ]:
ds.to_xarray()

(section-analysis)=
### 2. Analysis

#### Subsections
Describe what is done in this step/section and what the `code` in the cell does (if code is included).

(section-3)=
### 3. Section 3 title

#### Subsections
Describe what is done in this step/section and what the `code` in the cell does (if code is included).

(section-4)=
### 4. Sel examples

For reference

In [ ]:
domain = ekp.geo.domains.union(["United Kingdom", "Ireland"], name="UK & Ireland")

In [ ]:
oneday = ds.sel(date=20240505)

In [ ]:
ekp.quickplot(oneday, domain=domain, units="celsius")

In [ ]:
temperature = ds.sel(variable="Temperature_Air_2m_Max_24h")
chart = ekp.Map(domain=domain)
chart.contourf(temperature.sel(date=20240505), units="celsius", levels={"step": 2})
chart.land()
chart.coastlines()
chart.gridlines()
chart.legend()
chart.title()
chart.show()

In [ ]:
rad = ds.sel(variable="Solar_Radiation_Flux")
chart = ekp.Map(domain=domain)
chart.contourf(rad.sel(date=20240505))
chart.land()
chart.coastlines()
chart.gridlines()
chart.legend()
chart.title()
chart.show()

(section-5)=
### 5. Section 5 title 

#### Results Subsections
Describe what is done in this step/section and what the `code` in the cell does (if code is included). 

If this is the **results section**, we expect the final plots to be created here with a description of how to interpret them, and what information can be extracted for the specific use case and user question. The information in the 'quality assessment statement' should be derived here. 

## ℹ️ If you want to know more

### Key resources

List some key resources related to this assessment. E.g. CDS entries, applications, dataset documentation, external pages.
Also list any code libraries used (if applicable).

Code libraries used:
* Earthkit
  * [earthkit-data](https://github.com/ecmwf/earthkit)
  * [earthkit-plots](https://github.com/ecmwf/earthkit-plots)

### References
[[CDS AgERA5]](https://doi.org/10.24381/cds.6c68c9bb) Boogaard, H., Schubert, J., De Wit, A., Lazebnik, J., Hutjes, R., Van der Grijn, G., (2020): Agrometeorological indicators from 1979 to present derived from reanalysis. Copernicus Climate Change Service (C3S) Climate Data Store (CDS). DOI: 10.24381/cds.6c68c9bb (Accessed on DD-MMM-YYYY)